In [14]:
from pandas_datareader import data as pdr
import numpy as np
import pandas as pd
from datetime import date
import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pylab import rcParams
from random import sample, choice
from numba import njit
from itertools import combinations, chain
from collections import defaultdict
import seaborn as sns
import plotly as py
import plotly.graph_objs as go
import calendar
%matplotlib inline
sns.set()


In [15]:
#Import data
'''Note:Yahoo Fiance SP&500 data reached back to 1950. However, error "mktime argument out of range" happens when downloading 
data before 1970. This seems to be related to windows platform not supporting dates previous to Unix epoch. We will manaually 
download the data between 1950 and 1970 and will combine wiht the dataframe obtained from the DataReader. '''

#Dataframe with daily S&P500 data from 1970 to current date.
start_date=datetime.date(1970, 1, 1)
end_date=datetime.datetime.now()
ticker='^GSPC'
data_source='yahoo'

SP500_day=pdr.DataReader(ticker,start=start_date,end=end_date,data_source=data_source)

#Dataframe with daily S&P500 data from initial (1950) to 1970
SP500_temp=pd.read_csv(r'C:\Users\lcast\Desktop\DS\Projects\SP Timeseries\GSPC.csv',parse_dates=['Date'],index_col='Date')

#Joining both datafarmes in a single one with all the information from 1950 to current date
SP500_day=SP500_temp[:'1969'].append(SP500_day)

del SP500_temp

#Close and Adjusted Close are the same for this ticker, so we remove Adjusted Closing column and keep Closing.
SP500_day=SP500_day.drop(columns='Adj Close')
print(SP500_day.head(10))


                 Open       High        Low      Close   Volume
Date                                                           
1950-01-03  16.660000  16.660000  16.660000  16.660000  1260000
1950-01-04  16.850000  16.850000  16.850000  16.850000  1890000
1950-01-05  16.930000  16.930000  16.930000  16.930000  2550000
1950-01-06  16.980000  16.980000  16.980000  16.980000  2010000
1950-01-09  17.080000  17.080000  17.080000  17.080000  2520000
1950-01-10  17.030001  17.030001  17.030001  17.030001  2160000
1950-01-11  17.090000  17.090000  17.090000  17.090000  2630000
1950-01-12  16.760000  16.760000  16.760000  16.760000  2970000
1950-01-13  16.670000  16.670000  16.670000  16.670000  3330000
1950-01-16  16.719999  16.719999  16.719999  16.719999  1460000


In [16]:
#Just by looking in to the chart, it;s clear that stay in the market... blabalala
#Visualization to show historical trend. Always be in the market. Break-down 
#by different rolling windows

#Resample form day to a year for the historic trend
SP500_year=SP500_day.asfreq('Y',method='ffill')
SP500_data=go.Scatter(x=SP500_year.index.year,y=SP500_year['Close'])
layout=go.Layout(title='S&P500 historic', xaxis=dict(title='Year'),yaxis=dict(title='Price'))
fig=go.Figure(data=[SP500_data],layout=layout)
fig.update_layout(hovermode="x")
fig.show()

In [17]:
#Grow from 16.66 at the beginning og 1950 to 3244.67, almost 19376 %. Constant grow except between 1998and 2011, which emcompasses 
#the dotcom crisis and real estate crisis of 2009. 
#Let's take a look to different cumulative periods:
SP500_year['Yearly_Change']=SP500_year.Close.pct_change()

#Manually calculate the first year % change
SP500_year['Yearly_Change'][0]=SP500_year.loc['1950-12-31']['Close']/SP500_day.loc['1950-1-3']['Open']-1


SP500_year['Cumulative']= (1+SP500_year['Yearly_Change']).cumprod()-1
SP500_year['Cumulative_roll3']= ((1+SP500_year['Yearly_Change']).rolling(3)).apply(np.prod)-1
SP500_year['Cumulative_roll5']= ((1+SP500_year['Yearly_Change']).rolling(5)).apply(np.prod)-1
SP500_year['Cumulative_roll10']= ((1+SP500_year['Yearly_Change']).rolling(10)).apply(np.prod)-1
SP500_year['Cumulative_roll15']= ((1+SP500_year['Yearly_Change']).rolling(15)).apply(np.prod)-1
SP500_year['Cumulative_roll20']= ((1+SP500_year['Yearly_Change']).rolling(20)).apply(np.prod)-1





C:\Users\lcast\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
fig=go.Figure()
fig.add_trace(
    go.Bar(x=SP500_year.index.year,y=SP500_year['Yearly_Change'],
           visible=True, 
            hovertemplate ='%{y:.2%} <extra></extra>',
           marker_color=np.where(SP500_year['Yearly_Change']<0,'red','darkblue')
          )
        )
                     
buttons=[]
                         
cols= ['Yearly_Change','Cumulative_roll3','Cumulative_roll5','Cumulative_roll10','Cumulative_roll15',
       'Cumulative_roll20']   
cols_rename=['1 year','3 years', '5 years','10 years','15 years','20 years']

for col, col2 in zip(cols,cols_rename):
        buttons.append(dict(method='restyle',
                            label=col2,
                            visible=True,
                            args=[{'y':[SP500_year[col]],
                                   'x':[SP500_year.index.year],
                                    'type':'bar','marker.color':[np.where(SP500_year[col]<0,'red','darkblue')]
                                  }]
                           ))

fig.update_layout(
    updatemenus=[
        dict(type='buttons',
             direction='left',
            buttons=buttons
            ,x=0,xanchor='left',y=1.12
            )
        ])   

fig.update_layout(
    title=dict(text='Cumulative % change by rolling period', yref="container",xref="container",x=0,y=0.99),
    xaxis=dict(               
        title="Year(s)"        
    ),
    font=dict(color='black'),
     yaxis=dict(               
        title="% return"        
    ),
    annotations=[
        dict(text="Rolling period:", showarrow=False,
                             x=0, y=1.18, yref="paper",xref="paper")
    ]
)

fig.update_layout(hovermode="x")


#hoverlabel=dict(bgcolor='white')




In [22]:
#'The table below shows some basic metrics for different windows. The average yearly returns is 9.45% and around 73% of the years 
#'have possitive returns. When the window is increased to 10 years, the percentage of 10 year periods is reduced to 7/61=11% and 
#'for 15 years and 20 years periods there simply are not any with negative returns. Not bad considering that this includes crisis such as...
#'an confrims the idea the stay the curse is not a bad idea.

##Count of intervals,number of negatives,Table mean, max gain, min gain
print('Stats for differents rolling windows between 1950 and 2019')
negative_returns=lambda x:sum(x<0)
negative_returns.__name__='# Negative returns' #to name the lambda used in the agg
SP500_year.agg({'Yearly_Change':['count',negative_returns,'mean','std','max','min'],
                'Cumulative_roll3':['count',negative_returns,'mean','std','max','min'],
               'Cumulative_roll5':['count',negative_returns,'mean','std','max','min'],
               'Cumulative_roll10':['count',negative_returns,'mean','std','max','min'],
               'Cumulative_roll15':['count',negative_returns,'mean','std','max','min'],
               'Cumulative_roll20':['count',negative_returns,'mean','std','max','min']
               }).rename(columns={'Yearly_Change': '1 year', 'Cumulative_roll3':'3 years','Cumulative_roll5':'5 years',
               'Cumulative_roll10':'10 years','Cumulative_roll15':'15 years','Cumulative_roll20':'20 years'
                }).transpose().style.format({
                'count':'{0:,.0f}', '# Negative returns': '{0:,.0f}', 'mean': '{:.2%}','std': '{:.2%}','max':'{:.2%}','min':'{:.2%}'})


######INCLUDE std

Stats for differents rolling windows between 1950 and 2019


,count,# Negative returns,mean,std,max,min
1 year,70,19,9.15%,16.53%,45.02%,-38.49%
3 years,68,10,27.88%,29.79%,111.30%,-40.12%
5 years,66,14,50.42%,48.05%,219.91%,-25.53%
10 years,61,7,116.40%,90.70%,342.61%,-26.52%
15 years,56,0,211.78%,172.55%,778.53%,14.48%
20 years,51,0,347.34%,276.76%,1261.17%,71.28%


In [24]:
#1. Out of the market several periods

#1.1 Sell in May and go away
""""Sell in May and go away" is a well-known financial-world adage. It is based on the historical 
underperformance of some stocks in the "summery" six-month period commencing in May and ending in October,
 compared to the "wintery" six-month period from November to April. If an investor follows the this strategy, 
 they would divest their equity holdings in May (or at least, the late spring) and invest again in November
 (or the mid-autumn).

Some investors find this strategy more rewarding than staying in the equity markets throughout the year. 
"""
#Compare returns of the two halfs: May-October vs Jan to April+ Nov and Dec
#Create dataframe wth monthly information
SP500_month=SP500_day.asfreq('M',method='ffill')
SP500_month['Monthly_Change']=SP500_month.Close.pct_change()


#Simulate we are out of the market between May and October assign 0% return. Same for the other half
SP500_month['Monthly_Change_no_5-10']=SP500_month['Monthly_Change']
SP500_month.loc[SP500_month['Monthly_Change_no_5-10'].index.month.isin([5,6,7,8,9,10]),'Monthly_Change_no_5-10']=0
SP500_month['Monthly_Change_no_11-4']=SP500_month['Monthly_Change']
SP500_month.loc[SP500_month['Monthly_Change_no_11-4'].index.month.isin([1,2,3,4,11,12]),'Monthly_Change_no_11-4']=0

SP500_year['Yearly_Change_no_5-10']=((1+SP500_month['Monthly_Change_no_5-10']).rolling(12)).apply(np.prod)-1
SP500_year['Yearly_Change_no_11-4']=((1+SP500_month['Monthly_Change_no_11-4']).rolling(12)).apply(np.prod)-1


SP500_year[['Yearly_Change','Yearly_Change_no_5-10','Yearly_Change_no_11-4']].agg(['count',negative_returns,'mean','max','min']
            ).rename(columns={'Yearly_Change': 'Entire year', 'Yearly_Change_no_5-10':'November through April',
               'Yearly_Change_no_11-4':'May through October'}).transpose().style.format({'count':'{0:,.0f}', 
            '# Negative returns': '{0:,.0f}', 'mean': '{:.2%}','max':'{:.2%}','min':'{:.2%}'})



,count,# Negative returns,mean,max,min
Entire year,70,19,9.15%,45.02%,-38.49%
November through April,69,20,7.10%,29.37%,-18.37%
May through October,70,25,1.51%,19.93%,-30.08%


In [25]:
tot_years_sell=len(SP500_year[SP500_year['Yearly_Change_no_5-10']>SP500_year['Yearly_Change']])
tot_years=len(SP500_year)

print('There are {} years out of the total of {} when a sell in may strategy outperforms the hold strategy.'.format(tot_years_sell,tot_years))
 


There are 25 years out of the total of 70 when a sell in may strategy outperforms the hold strategy.


In [26]:
#Now, has it been this behaviour the same during the whole series? We will check by a decomposition by decades and also with a chart
#Visually, we can appreciate the same notion of same patern across the time in the following chart
fig=go.Figure()
fig.add_trace(
    go.Bar(x=SP500_year.index.year,y=SP500_year['Yearly_Change_no_5-10'],
           visible=True, 
            hovertemplate ='%{y:.2%} <extra></extra>',
           marker_color=np.where(SP500_year['Yearly_Change_no_5-10']<0,'red','darkblue')
          )
        )
fig.update_layout(hovermode="x")

#We don't see a different pattern across all the historic timeframe
#Sell in may decomposition


In [31]:
SP500_year[['Yearly_Change','Yearly_Change_no_5-10','Yearly_Change_no_11-4']].groupby((SP500_year.index.year//10)*10).agg(['count'
            ,negative_returns,'mean','max','min']).rename(columns={'Date':'Decade','Yearly_Change': 'Total year', 
            'Yearly_Change_no_5-10':'November to April','Yearly_Change_no_11-4':'May to October'}).style.format(
            {('Total year','count'):'{0:,.0f}', ('Total year','# Negative returns'): '{0:,.0f}', ('Total year','mean'): '{:.2%}',
             ('Total year','max'):'{:.2%}',('Total year','min'):'{:.2%}',('November to April','count'):'{0:,.0f}', 
             ('November to April','# Negative returns'): '{0:,.0f}', ('November to April','mean'): '{:.2%}',
             ('November to April','max'):'{:.2%}',('November to April','min'):'{:.2%}',('May to October','count'):
             '{0:,.0f}', ('May to October','# Negative returns'): '{0:,.0f}', ('May to October','mean'): '{:.2%}',
             ('May to October','max'):'{:.2%}',('May to October','min'):'{:.2%}'
            })


In [32]:
#Conclusion: Backtesting with historic data shows that investment returns can be improved by employing a seasonal 
#switching strategy.Doesn't hurt to stay but opportunity cost of switching to an alternate investment. 
#Only need to beat 1.51% but also need to consider the taxes costs

SP500_month['Cumulative']= (1+SP500_month['Monthly_Change']).cumprod()-1
SP500_month['Cumulative_no_5-10']= (1+SP500_month['Monthly_Change_no_5-10']).cumprod()-1
SP500_month['Cumulative_no_11-4']= (1+SP500_month['Monthly_Change_no_11-4']).cumprod()-1
SP500_year['Cumulative_no_5-10']=SP500_month['Cumulative_no_5-10'].asfreq('Y',method='ffill')
SP500_year['Cumulative_no_11-4']=SP500_month['Cumulative_no_11-4'].asfreq('Y',method='ffill')



fig = go.Figure()
fig.add_trace(go.Scatter(x=SP500_year.index.year, y=SP500_year['Cumulative'],
                    mode='lines',name='Full year return',hovertemplate ='%{y:.2%} <extra></extra>'))
fig.add_trace(go.Scatter(x=SP500_year.index.year, y=SP500_year['Cumulative_no_5-10'],
                    mode='lines',name='Nov to Apr return',hovertemplate ='%{y:.2%} <extra></extra>'))
fig.add_trace(go.Scatter(x=SP500_year.index.year, y=SP500_year['Cumulative_no_11-4'],
                    mode='lines', name='May to Oct return',hovertemplate ='%{y:.2%} <extra></extra>',))

fig.layout=go.Layout(title='Comparison full-Sell in May and go away', xaxis=dict(title='Year',rangeslider=dict(visible=True)),
                     yaxis=dict(title='Cumulative return',tickformat='%'),hovermode="x")
fig.show()



In [33]:
#####1.b - Other times############################################################################
#Count the minimum each year and sum total total

#Initial EDA to identify historical negative months
SP500_month_avg=SP500_month.groupby(SP500_month.index.month).mean()
fig=go.Figure()
fig.add_trace(
    go.Bar(x=SP500_month_avg.index,y=SP500_month_avg['Monthly_Change'],
           visible=True, hovertemplate ='%{y:.2%} <extra></extra>',
           marker_color=np.where(SP500_month_avg['Monthly_Change']<0,'red','darkblue')
          )
        )
fig.layout=go.Layout(title='S&P500 average monthly return {}-{}'.format(SP500_month.index.year.min(),
                    SP500_month.index.year.max()), xaxis=dict(title='Month',tickmode='linear'),yaxis=dict(title='% return',
                    tickformat='.2%'),hovermode="x")
fig.show()





In [34]:
SP500_month[SP500_month['Monthly_Change']<0].groupby(SP500_month[SP500_month['Monthly_Change']<0].index.month)['Monthly_Change'].count()

Date
1     28
2     32
3     26
4     20
5     29
6     33
7     30
8     32
9     37
10    28
11    22
12    18
Name: Monthly_Change, dtype: int64

In [36]:
SP500_month['Monthly_Change_no_8-9']=SP500_month['Monthly_Change']
SP500_month.loc[SP500_month['Monthly_Change_no_8-9'].index.month.isin([8,9]),'Monthly_Change_no_8-9']=0
SP500_year['Yearly_Change_no_8-9']=((1+SP500_month['Monthly_Change_no_8-9']).rolling(12)).apply(np.prod)-1

SP500_month['Monthly_Change_only_8-9']=SP500_month['Monthly_Change']
SP500_month.loc[SP500_month['Monthly_Change_only_8-9'].index.month.isin([1,2,3,4,5,6,7,10,11,12]),'Monthly_Change_only_8-9']=0
SP500_year['Yearly_Change_only_8-9']=((1+SP500_month['Monthly_Change_only_8-9']).rolling(12)).apply(np.prod)-1

SP500_month['Monthly_Change_only_9']=SP500_month['Monthly_Change']
SP500_month.loc[SP500_month['Monthly_Change_only_9'].index.month.isin([1,2,3,4,5,6,7,8,10,11,12]),'Monthly_Change_only_9']=0
SP500_year['Yearly_Change_only_9']=((1+SP500_month['Monthly_Change_only_9']).rolling(12)).apply(np.prod)-1

SP500_year[['Yearly_Change','Yearly_Change_no_8-9','Yearly_Change_only_8-9','Yearly_Change_only_9']].agg(['count',negative_returns,'mean','max','min']
            ).rename(columns={'Yearly_Change': 'Total year', 'Yearly_Change_no_8-9':'No August and Septemeber',
                              'Yearly_Change_only_8-9':'August and Septemeber','Yearly_Change_only_9':'September'
            }).transpose().style.format({'count':'{0:,.0f}', 
            '# Negative returns': '{0:,.0f}', 'mean': '{:.2%}','max':'{:.2%}','min':'{:.2%}'})

,count,# Negative returns,mean,max,min
Total year,70,19,9.15%,45.02%,-38.49%
No August and Septemeber,69,17,9.53%,39.58%,-33.16%
August and Septemeber,70,30,-0.50%,12.45%,-19.88%
September,70,37,-0.43%,8.76%,-11.93%


In [41]:
tot_years_sell_8_9=len(SP500_year[SP500_year['Yearly_Change_no_8-9']>SP500_year['Yearly_Change']])

print('There are {} years out of the total of {} when a sell in August and September strategy outperforms the hold strategy.'.format(tot_years_sell_8_9,tot_years))


There are 30 years out of the total of 70 when a sell in August and September strategy outperforms the hold strategy.


In [43]:
SP500_year[['Yearly_Change','Yearly_Change_no_8-9','Yearly_Change_only_8-9']].groupby((SP500_year.index.year//10)*10).agg(['count'
            ,negative_returns,'mean','max','min']).rename(columns={'Date':'Decade','Yearly_Change': 'Total year', 
            'Yearly_Change_no_8-9':'No August and Septemeber','Yearly_Change_only_8-9':'August and Septemeber'}).style.format(
            {('Total year','count'):'{0:,.0f}', ('Total year','# Negative returns'): '{0:,.0f}', ('Total year','mean'): '{:.2%}',
             ('Total year','max'):'{:.2%}',('Total year','min'):'{:.2%}',('No August and Septemeber','count'):'{0:,.0f}', 
             ('No August and Septemeber','# Negative returns'): '{0:,.0f}', ('No August and Septemeber','mean'): '{:.2%}',
             ('No August and Septemeber','max'):'{:.2%}',('No August and Septemeber','min'):'{:.2%}',
             ('August and Septemeber','count'):'{0:,.0f}', ('August and Septemeber','# Negative returns'): '{0:,.0f}', 
             ('August and Septemeber','mean'): '{:.2%}',('August and Septemeber','max'):'{:.2%}',
             ('August and Septemeber','min'):'{:.2%}'
            })

In [47]:
SP500_month['Cumulative_no_8-9']= (1+SP500_month['Monthly_Change_no_8-9']).cumprod()-1
SP500_month['Cumulative_only_8-9']= (1+SP500_month['Monthly_Change_only_8-9']).cumprod()-1
SP500_year['Cumulative_no_8-9']=SP500_month['Cumulative_no_8-9'].asfreq('Y',method='ffill')
SP500_year['Cumulative_only_8-9']=SP500_month['Cumulative_only_8-9'].asfreq('Y',method='ffill')



fig = go.Figure()
fig.add_trace(go.Scatter(x=SP500_year.index.year, y=SP500_year['Cumulative'],
                    mode='lines',name='Full year',hovertemplate ='%{y:.2%} <extra></extra>'))
fig.add_trace(go.Scatter(x=SP500_year.index.year, y=SP500_year['Cumulative_no_8-9'],
                    mode='lines',name='Without August and September',hovertemplate ='%{y:.2%} <extra></extra>'))
fig.add_trace(go.Scatter(x=SP500_year.index.year, y=SP500_year['Cumulative_only_8-9'],
                    mode='lines', name='August and September',hovertemplate ='%{y:.2%} <extra></extra>',))

fig.layout=go.Layout(title='Comparison total vs Sell August and September', xaxis=dict(title='Year',rangeslider=dict(visible=True)),
                     yaxis=dict(title='return',tickformat='%'),hovermode="x" )

fig.show()



In [48]:
'''DCA. Given an initial balance and a periodical contribution amount, calculate the final balance
on a period (full series, 5 year windows, 10 year window...) using different strategies.
strategies on when to invest :
    1. One time contribution per year: Lump sum of the contribution at the begining of the year.
    2. Monthly contributon: Split the total annual contribution among the 12 months of the year.
    3. Semester contribution: Split the total annual contribution in 2 and invest in January and July
    4. Quarterly Contributio: Split the total annual contribution in 4 and invest in 
    January (1Q), April (2Q),  July (3Q), October (4Q)
    5. Every two years: Lump sump amount at the begining of every other year.'''


SP500_DCA=pd.DataFrame()
SP500_DCA=SP500_day.asfreq('M',method='ffill').loc[:,['Close']]
SP500_DCA['Monthly_Change']=SP500_DCA.Close.pct_change()


In [49]:
@njit
def calculate_balance (a,b,c):
#a:current balance;b:percent change from previous period;c:contribution
    balance=[]
    balance.append(a[0])
    for i in range (1,len(a)):
       a[i]=a[i-1]*(1+b[i])+c[i]
       balance.append(a[i])
    return balance

In [50]:
#CHECK Initial contribution. 
#CHECK CAPTURING THE PERCENTAGE OF JANUARY

Initial_amount=10000
Annual_Contribution=12000

#Baseline: No investment
SP500_DCA['Balance']=0
SP500_DCA['Deposit']=0
SP500_DCA.loc[SP500_DCA.index.month.isin([1]),'Deposit']=Annual_Contribution
SP500_DCA.loc[SP500_DCA.index.min(),'Balance'] =Initial_amount+SP500_DCA['Deposit'][0]

SP500_DCA['Balance']=calculate_balance(SP500_DCA['Balance'].values.astype(float),
                                            np.array([0]*len(SP500_DCA['Monthly_Change'])),
                                            SP500_DCA['Deposit'].values.astype(float))

#Scenario 1. Annual lump sum contribution.
SP500_DCA['Balance_Annual']=0
SP500_DCA['Deposit_Annual']=0

#Updating the anual contribution, in this case at the beginning (January) of each year of the series
SP500_DCA.loc[SP500_DCA.index.month.isin([1]),'Deposit_Annual']=Annual_Contribution
SP500_DCA.loc[SP500_DCA.index.min(),'Balance_Annual'] =Initial_amount+SP500_DCA['Deposit_Annual'][0]
SP500_DCA['Balance_Annual']=calculate_balance(SP500_DCA['Balance_Annual'].values.astype(float),
                                            SP500_DCA['Monthly_Change'].fillna(1).values,
                                            SP500_DCA['Deposit_Annual'].values.astype(float))

#Scenario 2. Monthly contributon.
SP500_DCA['Balance_Monthly']=0
SP500_DCA['Deposit_Monthly']=0
SP500_DCA.loc[SP500_DCA.index.month.isin(range(1,13)),'Deposit_Monthly']=Annual_Contribution/12
SP500_DCA.loc[SP500_DCA.index.min(),'Balance_Monthly'] =Initial_amount+SP500_DCA['Deposit_Monthly'][0]
SP500_DCA['Balance_Monthly']=calculate_balance(SP500_DCA['Balance_Monthly'].values.astype(float),
                                         SP500_DCA['Monthly_Change'].fillna(1).values,
                                        SP500_DCA['Deposit_Monthly'].values.astype(float))


#Scenario 3. Semester contribution.
SP500_DCA['Balance_Semester']=0
SP500_DCA['Deposit_Semester']=0
SP500_DCA.loc[SP500_DCA.index.month.isin([1,6]),'Deposit_Semester']=Annual_Contribution/2
SP500_DCA.loc[SP500_DCA.index.min(),'Balance_Semester'] =Initial_amount+SP500_DCA['Deposit_Semester'][0]
SP500_DCA['Balance_Semester']=calculate_balance(SP500_DCA['Balance_Semester'].values.astype(float),
                                         SP500_DCA['Monthly_Change'].fillna(1).values,
                                         SP500_DCA['Deposit_Semester'].values.astype(float))


#Scenario 4. Quarterly contribution.
SP500_DCA['Balance_Quarter']=0
SP500_DCA['Deposit_Quarter']=0
SP500_DCA.loc[SP500_DCA.index.month.isin([1,4,7,10]),'Deposit_Quarter']=Annual_Contribution/4
SP500_DCA.loc[SP500_DCA.index.min(),'Balance_Quarter'] =Initial_amount+SP500_DCA['Deposit_Quarter'][0]
SP500_DCA['Balance_Quarter']=calculate_balance(SP500_DCA['Balance_Quarter'].values.astype(float),
                                         SP500_DCA['Monthly_Change'].fillna(1).values,
                                         SP500_DCA['Deposit_Quarter'].values.astype(float))

#Scenario 5. Two years lump sum
SP500_DCA['Balance_Biannual']=0
SP500_DCA['Deposit_Biannual']=0
SP500_DCA.loc[SP500_DCA.index.year.isin(range(SP500_DCA.index.year.min()+2,SP500_DCA.index.year.max()+1,2))&
              SP500_DCA.index.month.isin([1]),'Deposit_Biannual']=Annual_Contribution*2
SP500_DCA.loc[SP500_DCA.index.min(),'Balance_Biannual'] =Initial_amount+SP500_DCA['Deposit_Biannual'][0]
SP500_DCA['Balance_Biannual']=calculate_balance(SP500_DCA['Balance_Biannual'].values.astype(float),
                                         SP500_DCA['Monthly_Change'].fillna(1).values,
                                         SP500_DCA['Deposit_Biannual'].values.astype(float))


SP500_DCA.loc[SP500_DCA.index.max(),['Balance','Balance_Annual','Balance_Monthly','Balance_Semester','Balance_Quarter','Balance_Biannual']]

#SP500_DCA[['Balance_Annual','Balance_Monthly','Balance_Semester','Balance_Quarter','Balance_Biannual']].plot()
#plt.show()

Balance             8.620000e+05
Balance_Annual      2.611891e+07
Balance_Monthly     2.542098e+07
Balance_Semester    2.589041e+07
Balance_Quarter     2.552230e+07
Balance_Biannual    2.340507e+07
Name: 2020-05-31 00:00:00, dtype: float64

In [51]:
SP500_DCA.loc[SP500_DCA.index.min(),['Balance','Balance_Annual','Balance_Monthly','Balance_Semester','Balance_Quarter','Balance_Biannual']]

Balance             22000.0
Balance_Annual      22000.0
Balance_Monthly     11000.0
Balance_Semester    16000.0
Balance_Quarter     13000.0
Balance_Biannual    10000.0
Name: 1950-01-31 00:00:00, dtype: float64

In [52]:
SP500_DCA_year=SP500_DCA.asfreq('Y',method='ffill')

fig = go.Figure()

for col in list(['Balance','Balance_Annual','Balance_Monthly','Balance_Semester','Balance_Quarter','Balance_Biannual']):
     fig.add_trace(go.Scatter(x=SP500_DCA_year.index.year, y=SP500_DCA_year[col],
                             mode='lines',name=col))

fig.layout=go.Layout(title='DCA Strategies comparison', xaxis=dict(title='Year',rangeslider=dict(visible=True)),
                     yaxis=dict(title='Balance'),hovermode="x" )

fig.show()


In [63]:
#Evaluate in rolling indows
#TimeFrame 5,10,15,20
StartDate='1950'
EndDate='2019'
Frequency='AS' #alias for year start
Period=20


def period_intervals(StartDate,EndDate,Frequency,Period):
    #Create string to pass as parameter freq to the pandas function data_range
    #Generate a tuple with the pairs of intervals. 
    period_ini=pd.date_range(StartDate,str(int(EndDate)-Period),freq='AS').strftime('%Y-%m')
    period_end=pd.date_range(StartDate,str(int(EndDate)-Period),freq='AS').shift(12*Period,'M').strftime('%Y-%m')
    return period_ini,period_end

period_ini,period_end=period_intervals(StartDate,EndDate,Frequency,Period)



In [64]:
returns={}
for i,j in zip(period_ini,period_end):
    SP500_DCA_per=SP500_month.loc[i:j]
    Final_price=SP500_DCA_per['Close'][-1]
    SP500_DCA_per.loc[:,'Total%_Change']=Final_price/SP500_DCA_per['Close']-1
    for row in ['Balance_Annual','Deposit_Annual','Balance_Monthly','Deposit_Monthly','Balance_Semester','Deposit_Semester',
                'Balance_Quarter','Deposit_Quarter','Balance_Biannual','Deposit_Biannual']:
        SP500_DCA_per[row]=0 
    #Strategy 1
    SP500_DCA_per.loc[SP500_DCA_per.index.month.isin([1]),'Deposit_Annual']=Annual_Contribution
    SP500_DCA_per.loc[SP500_DCA_per.index.min(),'Deposit_Annual'] =SP500_DCA_per['Deposit_Annual'][0]+Initial_amount
    SP500_DCA_per.loc[:,'Balance_Annual'] =SP500_DCA_per['Deposit_Annual']* (SP500_DCA_per['Total%_Change']+1)
    #Strategy 2
    SP500_DCA_per.loc[SP500_DCA_per.index.month.isin(range(1,13)),'Deposit_Monthly']=Annual_Contribution/12
    SP500_DCA_per.loc[SP500_DCA_per.index.min(),'Deposit_Monthly'] =SP500_DCA_per['Deposit_Monthly'][0]+Initial_amount
    SP500_DCA_per.loc[:,'Balance_Monthly'] =SP500_DCA_per['Deposit_Monthly']* (SP500_DCA_per['Total%_Change']+1)
   
    #Strategy 3
    SP500_DCA_per.loc[SP500_DCA_per.index.month.isin([1,6]),'Deposit_Semester']=Annual_Contribution/2
    SP500_DCA_per.loc[SP500_DCA_per.index.min(),'Deposit_Semester'] =SP500_DCA_per['Deposit_Semester'][0]+Initial_amount
    SP500_DCA_per.loc[:,'Balance_Semester'] =SP500_DCA_per['Deposit_Semester']* (SP500_DCA_per['Total%_Change']+1)
        
    #Strategy 4
    SP500_DCA_per.loc[SP500_DCA_per.index.month.isin([1,4,7,10]),'Deposit_Quarter']=Annual_Contribution/4
    SP500_DCA_per.loc[SP500_DCA_per.index.min(),'Deposit_Quarter'] =SP500_DCA_per['Deposit_Quarter'][0]+Initial_amount
    SP500_DCA_per.loc[:,'Balance_Quarter'] =SP500_DCA_per['Deposit_Quarter']* (SP500_DCA_per['Total%_Change']+1)
    
    #Strategy 5
    SP500_DCA_per.loc[SP500_DCA_per.index.year.isin(range(SP500_DCA_per.index.year.min()+2,
                    SP500_DCA_per.index.year.max()+1,2))&SP500_DCA_per.index.month.isin([1]),'Deposit_Biannual']=Annual_Contribution*2
    SP500_DCA_per.loc[SP500_DCA_per.index.min(),'Deposit_Biannual'] =SP500_DCA_per['Deposit_Biannual'][0]+Initial_amount
    SP500_DCA_per.loc[:,'Balance_Biannual'] =SP500_DCA_per['Deposit_Biannual']* (SP500_DCA_per['Total%_Change']+1)

    returns[i+' to '+j]=[SP500_DCA_per['Balance_Annual'].sum(),SP500_DCA_per['Balance_Monthly'].sum(),
                      SP500_DCA_per['Balance_Semester'].sum(),SP500_DCA_per['Balance_Quarter'].sum(),
                      SP500_DCA_per['Balance_Biannual'].sum()]
    


C:\Users\lcast\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [56]:
SP500_DCA_per5=pd.DataFrame.from_dict(returns,orient='index')
SP500_DCA_per5.columns=['Balance_Annual','Balance_Monthly','Balance_Semester','Balance_Quarter','Balance_Biannual']
SP500_DCA_per5['Max']=SP500_DCA_per5.idxmax(axis=1)
SP500_DCA_per5['Max'].value_counts()

Balance_Annual      45
Balance_Semester    10
Balance_Monthly     10
Name: Max, dtype: int64

In [59]:
SP500_DCA_per10=pd.DataFrame.from_dict(returns,orient='index')
SP500_DCA_per10.columns=['Balance_Annual','Balance_Monthly','Balance_Semester','Balance_Quarter','Balance_Biannual']
SP500_DCA_per10['Max']=SP500_DCA_per10.idxmax(axis=1)
SP500_DCA_per10['Max'].value_counts()

Balance_Annual      43
Balance_Monthly     11
Balance_Semester     6
Name: Max, dtype: int64

In [62]:
SP500_DCA_per15=pd.DataFrame.from_dict(returns,orient='index')
SP500_DCA_per15.columns=['Balance_Annual','Balance_Monthly','Balance_Semester','Balance_Quarter','Balance_Biannual']
SP500_DCA_per15['Max']=SP500_DCA_per15.idxmax(axis=1)
SP500_DCA_per15['Max'].value_counts()

Balance_Annual      43
Balance_Semester     9
Balance_Monthly      3
Name: Max, dtype: int64

In [66]:
SP500_DCA_per20=pd.DataFrame.from_dict(returns,orient='index')
SP500_DCA_per20.columns=['Balance_Annual','Balance_Monthly','Balance_Semester','Balance_Quarter','Balance_Biannual']
SP500_DCA_per20['Max']=SP500_DCA_per20.idxmax(axis=1)
SP500_DCA_per20['Max'].value_counts()

Balance_Annual      43
Balance_Semester     7
Name: Max, dtype: int64

In [67]:
#Not disciplined and just invest Random number per year
#Try other way
SP500_DCA_random=SP500_month
SP500_DCA_random_totals=[]
list_years=SP500_DCA_random.index.year.unique()
final_price=SP500_DCA_random['Close'][-1]
#Calculate per each period the total return until the final
SP500_DCA_random.loc[:,'Total%_Change']=final_price/SP500_DCA_random['Close']-1


#random eahc year. Repeatt the experiment 10000
#@njit
for _ in range(10000):  
    SP500_DCA_random['Deposit']=0
    SP500_DCA_random['Balance']=0
    for year in list_years:
        #randomly select the number of months and whcih months the contributions are made
        months= np.random.choice(range(1,13),np.random.choice(range(1,13),1),replace=False)
        SP500_DCA_random.loc[(SP500_DCA_random.index.month.isin(months))& 
                             (SP500_DCA_random.index.year==year),'Deposit']=Annual_Contribution/len(months)
    SP500_DCA_random.loc[SP500_DCA_random.index.min(),'Deposit'] =Initial_amount+SP500_DCA_random['Deposit'][0]  
    SP500_DCA_random.loc[:,'Balance'] =SP500_DCA_random['Deposit']* (SP500_DCA_random['Total%_Change']+1)
    SP500_DCA_random_totals.append(SP500_DCA_random['Balance'].sum())
 



In [68]:
sum(SP500_DCA_random_totals)/len(SP500_DCA_random_totals)

25420316.896622423

In [70]:

np.sum(SP500_DCA_random_totals>SP500_DCA['Balance_Annual'][-1])/len(SP500_DCA_random_totals)
#np.sum(SP500_DCA_random_totals>SP500_DCA['Balance_Monthly'][-1])/len(SP500_DCA_random_totals)
#np.sum(SP500_DCA_random_totals>SP500_DCA['Balance_Semester'][-1])/len(SP500_DCA_random_totals)
#np.sum(SP500_DCA_random_totals>SP500_DCA['Balance_Quarter'][-1])/len(SP500_DCA_random_totals)
#np.sum(SP500_DCA_random_totals>SP500_DCA['Balance_Biannual'][-1])/len(SP500_DCA_random_totals)
    
    


0.0

In [71]:
#All combinations
SP500_DCA_combs=SP500_month
combs=[]
combs_totals=[]

for i in range (1,13):    
    combs.append(combinations(range(1,13), i)) #total of 4095 combinations

combs = list(chain.from_iterable(combs))

for i in range (0,len(combs)):  
    SP500_DCA_combs['Deposit']=0
    SP500_DCA_combs['Balance']=0
    SP500_DCA_combs.loc[SP500_DCA_combs.index.month.isin(combs[i]),'Deposit']=Annual_Contribution/len(combs[i])
    SP500_DCA_combs.loc[SP500_DCA_combs.index.min(),'Deposit'] =Initial_amount+SP500_DCA_combs['Deposit'][0]  
    SP500_DCA_combs.loc[:,'Balance'] =SP500_DCA_combs['Deposit']* (SP500_DCA_combs['Total%_Change']+1)
    combs_totals.append(SP500_DCA_combs['Balance'].sum())

SP500_DCA_combs_total=pd.concat([pd.DataFrame(combs),pd.Series(combs_totals)],axis=1)
SP500_DCA_combs_total.columns=['Month1','Month2','Month3','Month4','Month5','Month6','Month7','Month8','Month9','Month10','Month11',
              'Month12','Total']


    

In [72]:
#Avergae balance of combs
sum(combs_totals)/len(combs_totals)

25420982.228241898

In [74]:
#Percentage of monthly combinations over the Annual Lump sump strategy
np.sum(combs_totals>SP500_DCA['Balance_Annual'][-1])/len(combs_totals)*100

0.04884004884004884

In [75]:
#Worst results
SP500_DCA_combs_total.sort_values(['Total'],ascending=True).head(10)

,Month1,Month2,Month3,Month4,Month5,Month6,Month7,Month8,Month9,Month10,Month11,Month12,Total
11,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.430295e+07
77,11,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.452207e+07
76,10,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.473009e+07
297,10,11.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.473380e+07
10,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.474120e+07
287,7,11.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.475207e+07
67,7,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.475750e+07
293,8,11.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.476340e+07
71,8,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.477450e+07
296,9,11.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.477771e+07


In [76]:
#Best results
SP500_DCA_combs_total.sort_values('Total',ascending=False).head(10)

,Month1,Month2,Month3,Month4,Month5,Month6,Month7,Month8,Month9,Month10,Month11,Month12,Total
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.618597e+07
12,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.615244e+07
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.611891e+07
78,1,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.607992e+07
23,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.606042e+07
13,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.602690e+07
81,1,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.598893e+07
300,1,2.0,3.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.597542e+07
79,1,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.596863e+07
80,1,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.596850e+07


In [77]:
#Which ones improve the annual lump sump strategy
SP500_DCA_combs_total.loc[(SP500_DCA_combs_total['Total']> SP500_DCA['Balance_Annual'][-1])]



,Month1,Month2,Month3,Month4,Month5,Month6,Month7,Month8,Month9,Month10,Month11,Month12,Total
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.618597e+07
12,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.615244e+07


In [ ]:
#BUY THE DIP
##Buy the dip######################################################################################################3
###Cumulative###Buy the dip!
###What happens after xxx months of the dip
#We have established that is better to invest in the market rather than saving account. But what if in addition
#to DCA our dedicated amount, we would also decide to invest an extra surplus. We then 
#There are strategies that mean of 20 vs... But rememeber, we are keeping this simple, so we are going to evaluate something 
#more simple....
#Let's imagine to in addition to the annual contribution we have describe amount, we have an extra savings each month. 
#Now, we have two options: a. invest that amount sistematically b. accumulate thaose extra savings until the market drops
#certain percentage and the invest all the accumulated amouunt. 


In [78]:

#Function that takes as an input a percentage and identifies when declines in the price reach that percentage. The function 
#returns an aray that contains all the declines found for the stock series given as an input.
def find_drops (percentage,close):
#percentage:percentage of stock value drop;close:stock value at close,
    max=0
    drop=0
    b=[]
    for i in range (0,len(close)):
        if close[i]> max:
            max=close[i]
        decline=1-close[i]/max
        signal=0
        if decline>=percentage:
            signal=1
            max=close[i]
        b.append(signal)
    return b


#Extra deposit when the signal
def extra_deposit(a,extra):
#a:signal
    extra_deposit=0
    cum_deposit=[]
    for i in range (0,len(a)):
        extra_deposit+=extra
        cum_deposit.append(extra_deposit)
        if a[i]==1:
            extra_deposit=0
    return cum_deposit

In [ ]:
#Buying signas for drops of 5%, 10%, 15%, 20%,30%, 40%, %50
#Indicate there are not absolute drops
#Chart with buttons

In [79]:
#Did with days and weeks also, but no improvement
SP500_dip=SP500_day.asfreq('M',method='ffill')
SP500_dip['Monthly_Change']=SP500_dip['Close'].pct_change()
extra=200

#Baseline: Invest extra each monthc
SP500_dip['Balance_bench']=0
SP500_dip['Deposit_bench']=extra
SP500_dip.loc[SP500_dip.index.month.isin([1]),'Deposit_bench']+=Annual_Contribution
SP500_dip.loc[SP500_dip.index.min(),'Balance_bench'] =Initial_amount+SP500_dip['Deposit_bench'][0]

SP500_dip['Balance_bench']=calculate_balance(SP500_dip['Balance_bench'].values.astype(float),SP500_dip['Monthly_Change'].fillna(1).values,
                                           SP500_dip['Deposit_bench'].values.astype(float))



In [80]:
dips=[0.01,0.02,0.03,0.05,0.1,0.2,0.3,0.5]
dips_str= ['1%','2%','3%','5%','10%','20%','30%','50%']

for dip,dip_str in zip(dips,dips_str):
    signal='Signal_'+dip_str
    extra_='Extra_'+dip_str
    deposit_='Deposit_'+dip_str
    balance_='Balance_'+dip_str  
    SP500_dip[signal]=find_drops(dip,SP500_dip['Close'])
    SP500_dip[extra_]=extra_deposit(SP500_dip[signal],extra)
    SP500_dip[balance_]=0
    SP500_dip[deposit_]=SP500_dip[SP500_dip[signal]==1][extra_]
    SP500_dip[deposit_]=SP500_dip[deposit_].fillna(0)
    SP500_dip.loc[SP500_dip.index.month.isin([1]),deposit_]+=Annual_Contribution
    SP500_dip.loc[SP500_dip.index.min(),balance_] =Initial_amount+SP500_dip[deposit_][0]
    SP500_dip[balance_]=calculate_balance(SP500_dip[balance_].values.astype(float),SP500_dip['Monthly_Change'].fillna(1).values,
                                           SP500_dip[deposit_].values.astype(float))

    

In [81]:
 SP500_dip[['Balance_bench','Balance_1%','Balance_2%','Balance_3%','Balance_5%','Balance_10%',
           'Balance_20%','Balance_30%','Balance_50%']].iloc[-1]

Balance_bench    3.084601e+07
Balance_1%       3.075041e+07
Balance_2%       3.074055e+07
Balance_3%       3.070553e+07
Balance_5%       3.060847e+07
Balance_10%      3.057337e+07
Balance_20%      2.962799e+07
Balance_30%      2.912829e+07
Balance_50%      2.670699e+07
Name: 2020-05-31 00:00:00, dtype: float64

In [82]:
SP500_dip[['Signal_1%','Signal_2%','Signal_3%','Signal_5%','Signal_10%',
           'Signal_20%','Signal_30%','Signal_50%']].sum()

Signal_1%     262
Signal_2%     196
Signal_3%     157
Signal_5%     103
Signal_10%     39
Signal_20%     11
Signal_30%      5
Signal_50%      1
dtype: int64

In [ ]:
The time frame or length you choose for a moving average, also called the "look back period," can play a big role in how effective it is.
1) Move to inversely correlated assets * ,"safe" assets, or conduct portfolio hedging during LEI signaling accompanied by the price of the S&P 500 residing below it's 10 period monthly basis and 
2) move back into equity based assets on a S&P 500 price cross back above it's 10 period MA
    https://seekingalpha.com/article/4079476-10-market-timing-strategies-compared
    
    
    


In [ ]:
#Long version

fig.update_layout(
    updatemenus=[
        dict(type='buttons',
             direction='left',
            buttons=list([
                dict(
                    args=[{'y':[SP500_year['Yearly_Change']],
                                   'x':[SP500_year.index],
                                    'type':'bar'}],
                    label="1 year",
                    method="restyle"
                ),
                dict(
                    args=[{'y':[SP500_year['Cumulative_roll3']],
                                   'x':[SP500_year.index],
                                    'type':'bar'}],
                    label="3 years",
                    method="restyle"
                ),
                dict(
                    args=[{'y':[SP500_year['Cumulative_roll5']],
                                   'x':[SP500_year.index],
                                    'type':'bar'}],
                    label="5 years",
                    method="restyle"
                ),
                dict(
                    args=[{'y':[SP500_year['Cumulative_roll10']],
                                   'x':[SP500_year.index],
                                    'type':'bar'}],
                    label="10 years",
                    method="restyle"
                ),
                dict(
                    args=[{'y':[SP500_year['Cumulative_roll15']],
                                   'x':[SP500_year.index],
                                    'type':'bar'}],
                    label="15 years",
                    method="restyle"
                ),
                   dict(
                    args=[{'y':[SP500_year['Cumulative_roll20']],
                                   'x':[SP500_year.index],
                                    'type':'bar'}],
                    label="20 years",
                    method="restyle"
                   ),
            ]),x=0,xanchor='left',y=1.12
            )])   

fig.update_layout(
    title=dict(text='Cumulative % change by rolling period', yref="container",xref="container",x=0,y=0.99),
    xaxis=dict(               
        title="Year(s)"        
    ),
    font=dict(color='black'),
     yaxis=dict(               
        title="% Change"        
    ),
    annotations=[
        dict(text="Rolling period:", showarrow=False,
                             x=0, y=1.18, yref="paper",xref="paper")
    ]
)


In [ ]:
#Unused code to highligh minimum return months 

minimum_month=SP500_month.groupby([SP500_month.index.year])['Monthly_Change'].idxmin()

SP500_month.loc[minimum_month].index.month.value_counts()

month_names = dict(Jan=1, Feb=2, Mar=3,Apr=4,May=5,Jun=6,Jul=7,Aug=8,Sep=9,Oct=10,Nov=11,Dec=12) 



StartDate='1950'
EndDate='2010'
Frequency='AS' #alias for year start
Period=1

returns={}
avg_change={}

def period_intervals(StartDate,EndDate,Frequency,Period):
    #Create string to pass as parameter freq to the pandas function data_range
    frq=str(Period)+Frequency
    #Generate a tuple with the pairs of intervals. 
    period_ini=pd.date_range(StartDate,EndDate,freq=frq)#.strftime('%Y%')
    period_end=pd.date_range(StartDate,EndDate,freq=frq).shift(12*Period,'M')#.strftime('%Y%')
    return zip(period_ini,period_end);

period_ini=period_intervals(StartDate,EndDate,Frequency,Period)

for i,j in period_ini:
    
    avg_change[i.strftime('%Y')+'-'+j.strftime('%Y')] =GSPC_month.loc[i:j].Monthly_Change.groupby(GSPC_month.loc[i:j].index.month).mean()
    
    ##aaa=GSPC_month.loc[(GSPC_month.index.year>=i.year)&(GSPC_month.index.year<=j.year)]

aaa=pd.DataFrame.from_dict(avg_change,orient='index')





def highlight_min(s):    
    is_min = s.nsmallest(1).values
    return ['background-color: red' if v in is_min else '' for v in s]

#aaa.style.background_gradient(cmap='Blues',axis=1)
aaa.style.apply(highlight_min,axis=1)